## Version Description
In this version, we deploy a Hybrid PINN with adaptive Line Search in _single data point training_ mode, to predict 5 correction factors in TZ6, viz. for the MTR and 4 CLRs

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [3]:
# import required libraries
import pandas as pd
import numpy as np

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource,HoverTool
from bokeh.plotting import figure, show, output_file,output_notebook
from bokeh.io import export_svgs
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

### Data Acquisition

In [4]:
target_df = pd.read_csv('../data/TZ6_dataset.csv')
target_df.head()

,HoV,CAOLH_SumFlow,CAORH_SumFlow,LAOLH_SumFlow,LAORH_SumFlow,TZ6_Flow,MIXT,AMBT,AMBP,MIXP,R600_HD,R610_HS1,R611_HS1,R612_HS1,R613_HS1
0,A1,217.197005,222.443249,133.373448,136.693697,709.707399,293.150000,299.386667,101401.6000,2600.0000,148,131,136,120,120
1,A1,217.248929,223.698078,132.780168,135.684064,709.411240,293.150000,298.448667,101576.3000,2600.0000,149,131,136,120,120
2,A2,229.778639,226.831740,140.650844,143.718662,740.979885,298.101330,297.109024,102136.6035,2606.1928,152,131,136,114,120
3,A3,225.091032,227.788705,137.109226,142.122616,732.111579,294.390392,295.060027,103195.6642,2599.8998,154,131,136,120,120
4,A4,225.200989,222.557674,130.494552,135.055208,713.308423,293.150000,294.755833,102856.2000,2600.0000,148,145,153,130,130


In [5]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 15 columns):
HoV              35 non-null object
CAOLH_SumFlow    35 non-null float64
CAORH_SumFlow    35 non-null float64
LAOLH_SumFlow    35 non-null float64
LAORH_SumFlow    35 non-null float64
TZ6_Flow         35 non-null float64
MIXT             35 non-null float64
AMBT             35 non-null float64
AMBP             35 non-null float64
MIXP             35 non-null float64
R600_HD          35 non-null int64
R610_HS1         35 non-null int64
R611_HS1         35 non-null int64
R612_HS1         35 non-null int64
R613_HS1         35 non-null int64
dtypes: float64(9), int64(5), object(1)
memory usage: 4.2+ KB


In [6]:
# Define Restrictors and their parameters
MTR = 'R600_HD'
CLRS = ['R610_HS1','R611_HS1','R612_HS1','R613_HS1']

# Duct Areas
MTR_DuctArea = 0.03464 # in sq.m
MHR_Duct_Areas = {'R610_HS1':0.01767,'R611_HS1':0.01767,'R612_HS1':0.02461,'R613_HS1':0.02461}
restrictor_duct_areas = {'R600_HD':0.03464,'R610_HS1':0.01767,'R611_HS1':0.01767,'R612_HS1':0.02461,'R613_HS1':0.02461 }

## Define Initial Correction factor values from CFD
TZ6_zmix_cf = 1 # Start with initial c_f =1 
MTR_cf = 1.1664876228437
MHR_cf = {'R610_HS1':0.740773983383846,'R611_HS1':0.740205245723713,'R612_HS1':0.83879652680329,'R613_HS1':0.81775586281569}

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [7]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [8]:
_,target_df[MTR+'_Zeta1D'] = zip(*[SHR_Zeta_3D(1,dia,MTR_DuctArea,MTR_cf) for dia in target_df[MTR]])
# Gather all column names required for the input loss coefficients
zeta_col_names = [col for col in target_df.columns if 'Zeta1D' in col]
target_df[[MTR,'R600_HD_Zeta1D']].head()

,R600_HD,R600_HD_Zeta1D
0,148,4.949375
1,149,4.730821
2,152,4.127180
3,154,3.764504
4,148,4.949375


**For Multi Hole Restrictors:**<br>

    Area_Free = Number_of_Holes * A_Circular(Hole_Diameter)
    f0_f1 = Area_Free / Area_Overall
    l_cross = Thickness / Hole_Diameter
    phi = 0.25 + (0.535 * l_cross ^ 8) / (0.05 + l_cross ^ 7)
    tau = (2.4 - l_cross) * 10 ^ (-phi)
    Zeta_Multi_Hole = (0.5 * (1 - f0_f1) ^ 0.75 + tau * (1 - f0_f1) ^ 1.375 + (1 - f0_f1) ^ 2 + 0.02 * l_cross) / f0_f1 ^ 2

In [9]:
def MHR_Zeta_3D(nr_holes,hole_dia,MHR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Multi Hole Restrictors
    '''
    MHR_New_Area = nr_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MHR_New_Area/MHR_DuctArea
    l_cross = (0.00144*1000)/hole_dia
    phi = 0.25 + (0.535 * l_cross**8) / (0.05 + l_cross**7)
    tau = (2.4 - l_cross) * 10**(-phi)
    zeta_MHR_1D = (0.5 * (1 - f0_f1)**0.75 + tau * (1 - f0_f1)**1.375 + (1 - f0_f1)**2 + 0.02 * l_cross) / f0_f1**2 # 1D Zeta     
    zeta_MHR_3D = zeta_MHR_1D * cf # Zeta with 3D Correction Factor    
    return MHR_New_Area,zeta_MHR_3D

In [10]:
# Calculate Zeta for Multi-Hole Restrictors
for clr in CLRS:
    if (clr == 'R610_HS1') | (clr == 'R611_HS1'):
        hole_dia = 8
    else:
        hole_dia = 10
    MHR_nr_holes = target_df[clr].values
    _, target_df[clr+'_Zeta1D'] = zip(*[MHR_Zeta_3D(ele,hole_dia,MHR_Duct_Areas[clr],MHR_cf[clr]) for ele in MHR_nr_holes])

**Store Loss Coefficient Values and Names for all Restrictors**:

In [11]:
# Gather all column names required for the input loss coefficients
from TZ6_FDDN_Lib import zeta_values
idx = target_df.columns.get_loc("R600_HD_Zeta1D")
target_df.insert(loc=(idx-1), column='TZ6_zmix_Zeta1D', value=float(zeta_values[0]))
zeta_col_names = [col for col in target_df.columns if 'Zeta1D' in col]

### Training data Preparation

In [12]:
from TZ6_5CF import NormbyMax, PINN

In [13]:
features2scale = ['R600_HD_Zeta1D','R610_HS1_Zeta1D', 'R611_HS1_Zeta1D','R612_HS1_Zeta1D','R613_HS1_Zeta1D','LAORH_SumFlow','LAOLH_SumFlow','CAORH_SumFlow','CAOLH_SumFlow','TZ6_Flow']
features_max, df_rescaled = NormbyMax(target_df,features2scale)
print('Max value of all features:',features_max)

Max value of all features: {'R600_HD_Zeta1D': 5.4148802645163, 'R610_HS1_Zeta1D': 10.02106896516872, 'R611_HS1_Zeta1D': 9.143664364318942, 'R612_HS1_Zeta1D': 10.11135599670757, 'R613_HS1_Zeta1D': 9.85771922403693, 'LAORH_SumFlow': 145.81587100000004, 'LAOLH_SumFlow': 144.888603, 'CAORH_SumFlow': 241.999479, 'CAOLH_SumFlow': 238.392638, 'TZ6_Flow': 764.0123719999998}


In [14]:
features_max_df = pd.DataFrame(data = features_max, index=[0])
print('Max value of all input features for normalization')
features_max_df

Max value of all input features for normalization


,R600_HD_Zeta1D,R610_HS1_Zeta1D,R611_HS1_Zeta1D,R612_HS1_Zeta1D,R613_HS1_Zeta1D,LAORH_SumFlow,LAOLH_SumFlow,CAORH_SumFlow,CAOLH_SumFlow,TZ6_Flow
0,5.41488,10.021069,9.143664,10.111356,9.857719,145.815871,144.888603,241.999479,238.392638,764.012372


In [15]:
V_max_org = features_max_df[['TZ6_Flow','LAORH_SumFlow','LAOLH_SumFlow','CAORH_SumFlow','CAOLH_SumFlow']]
V_max_org

,TZ6_Flow,LAORH_SumFlow,LAOLH_SumFlow,CAORH_SumFlow,CAOLH_SumFlow
0,764.012372,145.815871,144.888603,241.999479,238.392638


In [16]:
# Store indices and values of train data
input_features = ['R600_HD_Zeta1D','R610_HS1_Zeta1D', 'R611_HS1_Zeta1D','R612_HS1_Zeta1D','R613_HS1_Zeta1D']
train_data = df_rescaled[input_features].iloc[[11,21]]
train_data_idx = train_data.index.values.tolist()
train_R600HD_series = target_df['R600_HD'].iloc[train_data_idx]
train_CLR610HN_series = target_df['R610_HS1'].iloc[train_data_idx]
train_CLR611HN_series = target_df['R611_HS1'].iloc[train_data_idx]
train_CLR612HN_series = target_df['R612_HS1'].iloc[train_data_idx]
train_CLR613HN_series = target_df['R613_HS1'].iloc[train_data_idx]
print("Number of training samples:", len(train_data),';\tTrain data Shape:',train_data.shape,'\nSCALED FEATURES:')
train_data

Number of training samples: 2 ;	Train data Shape: (2, 5) 
SCALED FEATURES:


,R600_HD_Zeta1D,R610_HS1_Zeta1D,R611_HS1_Zeta1D,R612_HS1_Zeta1D,R613_HS1_Zeta1D
11,0.695215,0.687613,0.611478,0.783311,0.696188
21,0.914032,0.749034,0.737240,0.783311,0.783311


### Predict $c_f$ with Physics Informed Neural Network

In [17]:
### Set the hyperparameters here ###
N_i = train_data.shape[1] # No. of feature columns in the dataframe
hl_dim = [5] ### IMPORTANT PARAMETER
output_nodes = 5
network_dim = (N_i,hl_dim,output_nodes)

epsi_factor = 0.01
MTR_epsi = np.float(MTR_cf * epsi_factor)
CLR610_epsi = np.float(MHR_cf['R610_HS1'] * epsi_factor)
CLR611_epsi = np.float(MHR_cf['R611_HS1'] * epsi_factor)
CLR612_epsi = np.float(MHR_cf['R612_HS1'] * epsi_factor)
CLR613_epsi = np.float(MHR_cf['R613_HS1'] * epsi_factor)
epsi_tuple = (MTR_epsi, CLR610_epsi, CLR611_epsi, CLR612_epsi, CLR613_epsi)
rparams_series_tuple = (train_R600HD_series,train_CLR610HN_series,train_CLR611HN_series,train_CLR612HN_series,train_CLR613HN_series)

#### Train data points sequentially - Single Data Point Training Mode

In [18]:
%%time
learning_rates = [0, 0.001] ### IMPORTANT PARAMETER 
network = PINN(epsi_tuple, activation_function_choice = 'linear', error_loss_choice = 'full') #cf_only , simplified                                                                                               
MSE_flowloss_hist_train, error_loss_hist_train, cf_hist_train, losses_hist = network.train_1p_seq_LS(learning_rates, network_dim, input_features, train_data[input_features], rparams_series_tuple, train_data_idx, restrictor_duct_areas, target_df, zeta_col_names, V_max_org, epochs = 15) 

Activation Function selected: linear
Loss Function & Delta-k Used: full

BEGIN Neural Network training for ['C6']
SEED value: 13
Nr of Hidden layers: 1 ;	Nr_Neurons in last hidden layer: 5
FIRST - INITIAL WEIGHTS & BIASES:
W1: [array([-0.319, 0.337, -0.0199, 0.202, 0.602]), array([0.238, 0.604, 0.385, 0.661, -0.468]), array([-0.353, -0.564, 0.252, -0.109, 0.409]), array([0.142, 0.0569, 0.962, 0.271, -0.012]), array([-0.44, 0.532, 0.426, -0.486, -0.0649])] <class 'numpy.ndarray'> Shape: (5, 5)
b1: [1.0, 1.0, 1.0, 1.0, 1.0] <class 'numpy.ndarray'> Shape: (5,)
W2: [array([0.106, -0.733, -0.124, 0.626, -0.722]), array([0.22, 0.846, -0.278, -0.203, 0.0972]), array([0.23, 0.178, -0.677, -0.342, 0.0453]), array([-0.142, 0.509, -0.144, -0.166, 0.827]), array([-0.0365, -0.438, -0.00107, -0.104, 0.318])] <class 'numpy.ndarray'> Shape: (5, 5)
b2: [1.0, 1.0, 1.0, 1.0, 1.0] <class 'numpy.ndarray'> Shape: (5,)

MTR-Cf: 1.752534565886352 ; CLR-610-Cf: 2.6501078066992982 ; CLR-611-Cf: -1.4080025292167

Flowrate_delta (FDDN - LTR): [array([2.24, -1.44, 1.16, 0.57, 1.95])] <class 'numpy.ndarray'> (1, 5)
f2: -0.15230056553681492 ;	lr2: 0.0017976714837297366 ;	MSE_FlowError: 2.5131148809530734 ;	Error: 0.07894664008066517
f2 < 0 -> Use Brent

MTR-Cf: 1.092276988686253 ; CLR-610-Cf: 0.8199041754378819 ; CLR-611-Cf: 1.063414853427107 ; CLR-612-Cf: 1.0686798645237718 ; CLR-613-Cf: 1.1042464044152547
Row ID: 11 	HoV: ['C6']
Flowrate_delta (FDDN - LTR): [array([2.1, -1.48, 1.15, 0.52, 1.91])] <class 'numpy.ndarray'> (1, 5)
zfunc, lr, f_lr: 0.0017306567925151342 -0.1518051633689264

MTR-Cf: 1.092276988686253 ; CLR-610-Cf: 0.8199041754378819 ; CLR-611-Cf: 1.063414853427107 ; CLR-612-Cf: 1.0686798645237718 ; CLR-613-Cf: 1.1042464044152547
Row ID: 11 	HoV: ['C6']
Flowrate_delta (FDDN - LTR): [array([2.1, -1.48, 1.15, 0.52, 1.91])] <class 'numpy.ndarray'> (1, 5)
lr2: 0.0017306567925151342 ;	MSE_FlowError: 2.3708233873260967 ;	Error: 0.07879843724581459

EPOCH: 4 ;	Progress: 26.7% ;	MSE Flowloss (T

Flowrate_delta (FDDN - LTR): [array([-0.626, -1.23, 0.379, -0.246, 0.467])] <class 'numpy.ndarray'> (1, 5)
f0: 0.5238980640746571 ;	lr0: 0 ;	MSE_FlowError: 0.46353039109917127 ;	Error: 0.06017689226663473

MTR-Cf: 1.1048622347095127 ; CLR-610-Cf: 0.7777069792706282 ; CLR-611-Cf: 1.0836124421523192 ; CLR-612-Cf: 1.0672105901121218 ; CLR-613-Cf: 1.12409172800858
Row ID: 11 	HoV: ['C6']
Flowrate_delta (FDDN - LTR): [array([0.369, -0.979, 0.544, 0.079, 0.725])] <class 'numpy.ndarray'> (1, 5)
f1: 0.169806260783746 ;	lr1: 0.001 ;	MSE_FlowError: 0.38434989583649537 ;	Error: 0.057407118784942805

Learning Rate 2: 0.0014795543393141984

MTR-Cf: 1.1014899941032263 ; CLR-610-Cf: 0.7758908181952677 ; CLR-611-Cf: 1.0840565947897378 ; CLR-612-Cf: 1.06667612724314 ; CLR-613-Cf: 1.1242029507368805
Row ID: 11 	HoV: ['C6']
Flowrate_delta (FDDN - LTR): [array([0.825, -0.863, 0.602, 0.237, 0.85])] <class 'numpy.ndarray'> (1, 5)
f2: -0.003986673584213184 ;	lr2: 0.0014795543393141984 ;	MSE_FlowError: 0.5132

Flowrate_delta (FDDN - LTR): [array([3.6, -0.509, 2.24, 0.946, 0.924])] <class 'numpy.ndarray'> (1, 5)
zfunc, lr, f_lr: 0.0032141219544694517 0.1168645038216588

MTR-Cf: 1.095783865458291 ; CLR-610-Cf: 0.9246369566798129 ; CLR-611-Cf: 1.0645425139956952 ; CLR-612-Cf: 1.0578449959716785 ; CLR-613-Cf: 0.9597720495695203
Row ID: 21 	HoV: ['J1']
Flowrate_delta (FDDN - LTR): [array([3.6, -0.509, 2.24, 0.946, 0.924])] <class 'numpy.ndarray'> (1, 5)
lr2: 0.0032141219544694517 ;	MSE_FlowError: 4.003236114161181 ;	Error: 0.07283422282338822

EPOCH: 2 ;	Progress: 13.3% ;	MSE Flowloss (Train):  [3.2448243716701697, 4.003236114161181] ;	Error_loss (Train):  [0.12967471520810575, 0.07283422282338822]

MTR-Cf: 1.095783865458291 ; CLR-610-Cf: 0.9246369566798129 ; CLR-611-Cf: 1.0645425139956952 ; CLR-612-Cf: 1.0578449959716785 ; CLR-613-Cf: 0.9597720495695203
Row ID: 21 	HoV: ['J1']
Flowrate_delta (FDDN - LTR): [array([3.6, -0.509, 2.24, 0.946, 0.924])] <class 'numpy.ndarray'> (1, 5)
f0: 4.20580293372

In [19]:
cf_df = pd.DataFrame(data=cf_hist_train)
losses = pd.DataFrame(data=losses_hist)

In [20]:
mseflow_last_loss = []
error_last_loss = []
for i in range(len(losses)):
    loss1 = losses.MSE_flowloss[i][-1]
    loss2 = losses.Error_loss[i][-1]
    mseflow_last_loss.append(loss1)
    error_last_loss.append(loss2)

cf_df['Final_MSELoss'] = mseflow_last_loss
cf_df['Final_ErrorLoss'] = error_last_loss

In [21]:
cf_df['MaxEpochs'] = None # Initialize Empty Column for storing epochs trained

for i in range(len(losses['HoV'])):
    nr_epochs_trained = len(losses['MSE_flowloss'][i])
    cf_df['MaxEpochs'].loc[i] = nr_epochs_trained
    
cf_df.to_csv('data_output/NN_5CF_Output.csv',index=False)

In [22]:
# cf_df.sort_values(by = [MTR,'HoV'], inplace = True)
# print('Max no. of epochs for:',cf_df['HoV'].iloc[[cf_df.MaxEpochs.idxmax()]].values)
# cf_df.iloc[[cf_df.MaxEpochs.idxmax()]]

### Plot Training Losses

In [23]:
from bokeh.palettes import Category20,Colorblind,Spectral,Set1,Set2,YlGnBu,RdPu
temp_list = []
bokeh_palettes = [Colorblind,Set1,YlGnBu,RdPu,Set2]
for palette in bokeh_palettes:
    for key in palette.keys():
        temp_list.append(palette[key])
    
color_palette =  [y for x in temp_list for y in x] # ['red','blue']
num_lines = len(losses.HoV) # no. of lines to draw
colors = color_palette[0:num_lines]
labels = losses.HoV.values.tolist()

In [24]:
hover = HoverTool(names = ['EpochPoints'],tooltips = [('Epoch', '@x'), ('Error','@y')])
hover.point_policy='snap_to_data'

**MSE FlowError Loss**

In [25]:
p1 = figure(title = 'MSE FlowLoss for TZ6_5CF TRAIN data points', width=1250)
for i in range(num_lines):
    x = list(range(1,len(losses['MSE_flowloss'][i])+1))
    y1 = losses['MSE_flowloss'][i]    
    p1.line(x, y1, line_width=2, color=colors[i], alpha=0.8, name = 'EpochPoints',legend='MSE Flow Loss for {}'.format(labels[i]))    
p1.yaxis.axis_label = "MSE FlowRate Loss"
p1.xaxis.axis_label = "Epochs"
p1.legend.click_policy="hide"
p1.add_tools(hover)
show(p1)

**Error Loss**

In [26]:
p2 = figure(title = 'Error (Loss) Function for TZ6_5CF TRAIN data', width=1250)
for i in range(num_lines):
    x = list(range(1,len(losses['Error_loss'][i])+1))     
    y2 = losses['Error_loss'][i]    
    p2.line(x, y2, line_width=2, color= colors[i], alpha=0.5, name = 'EpochPoints', legend='Error Loss for {}'.format(labels[i]))
    p2.circle(x, y2, line_width=2, color= colors[i], alpha=0.75, size = 8, name = 'EpochPoints', legend='Error Loss for {}'.format(labels[i]))
p2.yaxis.axis_label = "Error Loss"
p2.xaxis.axis_label = "Epochs"
p2.legend.click_policy="hide"
p2.xaxis.axis_label_text_font_size = '12pt'
p2.yaxis.axis_label_text_font_size = '12pt'
p2.xaxis.major_label_text_font_size= '11pt'
p2.xaxis.major_label_standoff = 15
p2.yaxis.major_label_text_font_size= '11pt'
p2.legend.label_text_font_size = '12pt'
p2.title.text_font_size = '12pt'
p2.add_tools(hover)
p2.output_backend = 'svg'
# export_svgs(p2, filename="plots/TZ6_5CF_ErrorLoss.svg")
show(p2)